# 📘 Common Function Library with Auto Parameter Estimation> Pre-built functions that eliminate manual parameter guessing⏱️ **15-20 minutes** | 📊 **Level: ●○○ Beginner** | 🏷️ **Feature Demo**---

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/05_feature_demos/function_library_demo.ipynb)


In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("✅ NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

## 🎯 Learning ObjectivesAfter this tutorial, you'll be able to:1. Use pre-built functions from `nlsq.functions`2. Leverage automatic parameter estimation with `p0='auto'`3. Fit common models without manual parameter guessing4. Apply functions to real-world curve fitting problems---

## 🔬 Feature Overview**What problem does this solve?**- Manual `p0` guessing is tedious and error-prone- Common functions are reimplemented repeatedly- Poor initial guesses lead to convergence failures**Available functions:**- `linear`: y = ax + b- `exponential_decay`: y = a·exp(-bx) + c- `exponential_growth`: y = a·exp(bx) + c- `gaussian`: y = a·exp(-(x-μ)²/(2σ²))- `sigmoid`: y = L/(1 + exp(-k(x-x₀))) + b- `power_law`: y = ax^b- `polynomial(degree)`: Creates polynomial of any degree**All functions include:**- Automatic p0 estimation- Reasonable default bounds- JAX/GPU acceleration- Comprehensive docstrings---

## Setup

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from nlsq import curve_fit, functions

## Example 1: Linear FunctionThe simplest case - no guessing needed!

In [ ]:
np.random.seed(42)
x = np.linspace(0, 10, 50)
y_true = 2.5 * x + 3.0
y = y_true + np.random.normal(0, 1.0, len(x))

# Fit without specifying p0 - automatic!
popt, pcov = curve_fit(functions.linear, x, y, p0='auto')

print(f'✓ Fitted: slope={popt[0]:.2f}, intercept={popt[1]:.2f}')
print('  True:   slope=2.50, intercept=3.00')

plt.scatter(x, y, alpha=0.5, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, functions.linear(x, *popt), 'r-', label='Fitted')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 2: Exponential DecayPerfect for radioactive decay, cooling, discharge, etc.

In [ ]:
np.random.seed(42)
x = np.linspace(0, 10, 100)
a_true, b_true, c_true = 100.0, 0.5, 10.0
y_true = a_true * np.exp(-b_true * x) + c_true
y = y_true + np.random.normal(0, 2.0, len(x))

# Automatic p0 estimation
popt, pcov = curve_fit(functions.exponential_decay, x, y, p0='auto')

# Calculate half-life
half_life_fitted = np.log(2) / popt[1]
half_life_true = np.log(2) / b_true

print(f'✓ Fitted: amplitude={popt[0]:.1f}, rate={popt[1]:.3f}, offset={popt[2]:.1f}')
print(f'  True:   amplitude={a_true:.1f}, rate={b_true:.3f}, offset={c_true:.1f}')
print(f'\n  Half-life (fitted): {half_life_fitted:.2f}')
print(f'  Half-life (true):   {half_life_true:.2f}')

plt.scatter(x, y, alpha=0.5, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, functions.exponential_decay(x, *popt), 'r-', label='Fitted')
plt.xlabel('Time')
plt.ylabel('Activity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 3: Gaussian PeakCommon in spectroscopy, chromatography, and image analysis.

In [ ]:
np.random.seed(42)
x = np.linspace(0, 20, 300)
amp_true, mu_true, sigma_true = 50.0, 12.0, 1.5
y_true = amp_true * np.exp(-((x - mu_true)**2) / (2*sigma_true**2))
y = y_true + np.random.normal(0, 1.0, len(x))

# Automatic p0 estimation
popt, pcov = curve_fit(functions.gaussian, x, y, p0='auto')

# Calculate FWHM (Full Width at Half Maximum)
fwhm_fitted = 2.355 * popt[2]
fwhm_true = 2.355 * sigma_true

print(f'✓ Fitted: amplitude={popt[0]:.1f}, center={popt[1]:.2f}, width={popt[2]:.2f}')
print(f'  True:   amplitude={amp_true:.1f}, center={mu_true:.2f}, width={sigma_true:.2f}')
print(f'\n  FWHM (fitted): {fwhm_fitted:.2f}')
print(f'  FWHM (true):   {fwhm_true:.2f}')

plt.scatter(x, y, alpha=0.3, s=10, label='Data')
plt.plot(x, y_true, 'g--', linewidth=2, label='True')
plt.plot(x, functions.gaussian(x, *popt), 'r-', linewidth=2, label='Fitted')
plt.axvline(popt[1], color='r', linestyle=':', alpha=0.5, label=f'Peak at {popt[1]:.1f}')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 4: Sigmoid (Dose-Response)Essential for pharmacology, biology, and neural networks.

In [ ]:
np.random.seed(42)
x = np.linspace(0, 10, 100)
L_true, x0_true, k_true, b_true = 100.0, 5.0, 1.5, 10.0
y_true = L_true / (1 + np.exp(-k_true * (x - x0_true))) + b_true
y = y_true + np.random.normal(0, 3.0, len(x))

# Automatic p0 estimation
popt, pcov = curve_fit(functions.sigmoid, x, y, p0='auto')

print(f'✓ Fitted: max={popt[0]:.1f}, EC50={popt[1]:.2f}, steepness={popt[2]:.2f}, baseline={popt[3]:.1f}')
print(f'  True:   max={L_true:.1f}, EC50={x0_true:.2f}, steepness={k_true:.2f}, baseline={b_true:.1f}')
print(f'\n  EC50 (half-maximal concentration): {popt[1]:.2f}')

plt.scatter(x, y, alpha=0.5, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, functions.sigmoid(x, *popt), 'r-', label='Fitted')
plt.axhline(popt[0]/2 + popt[3], color='gray', linestyle=':', alpha=0.5)
plt.axvline(popt[1], color='r', linestyle=':', alpha=0.5, label=f'EC50={popt[1]:.1f}')
plt.xlabel('Dose (concentration)')
plt.ylabel('Response')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 5: Power LawUsed in allometric scaling, physics, and economics.

In [ ]:
np.random.seed(42)
x = np.linspace(1, 100, 50)
a_true, b_true = 3.0, 0.75
y_true = a_true * x**b_true
y = y_true + np.random.normal(0, 0.5 * np.sqrt(y_true), len(x))

# Automatic p0 estimation
popt, pcov = curve_fit(functions.power_law, x, y, p0='auto')

print(f'✓ Fitted: prefactor={popt[0]:.2f}, exponent={popt[1]:.3f}')
print(f'  True:   prefactor={a_true:.2f}, exponent={b_true:.3f}')
print(f'\n  Scaling exponent: {popt[1]:.3f} (Kleiber\'s law predicts 0.75)')

plt.scatter(x, y, alpha=0.5, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, functions.power_law(x, *popt), 'r-', label='Fitted')
plt.xlabel('Body Mass (kg)')
plt.ylabel('Metabolic Rate')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 6: Polynomial (Quadratic)Create polynomials of any degree.

In [ ]:
# Create quadratic polynomial function
quadratic = functions.polynomial(2)

np.random.seed(42)
x = np.linspace(-5, 5, 60)
coeffs_true = [0.5, -2, 3]  # y = 0.5x² - 2x + 3
y_true = np.polyval(coeffs_true, x)
y = y_true + np.random.normal(0, 1.0, len(x))

# Automatic p0 estimation
popt, pcov = curve_fit(quadratic, x, y, p0='auto')

print(f'✓ Fitted: coeffs = [{popt[0]:.2f}, {popt[1]:.2f}, {popt[2]:.2f}]')
print('  True:   coeffs = [0.50, -2.00, 3.00]')
print(f'\n  Polynomial: y = {popt[0]:.2f}x² + {popt[1]:.2f}x + {popt[2]:.2f}')

# Find vertex
vertex_x = -popt[1] / (2 * popt[0])
vertex_y = np.polyval(popt, vertex_x)
print(f'  Vertex at ({vertex_x:.2f}, {vertex_y:.2f})')

plt.scatter(x, y, alpha=0.5, label='Data')
plt.plot(x, y_true, 'g--', label='True')
plt.plot(x, quadratic(x, *popt), 'r-', label='Fitted')
plt.plot(vertex_x, vertex_y, 'ro', markersize=8, label='Vertex')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.tight_layout()
plt.show()


## Example 7: Performance ComparisonAuto p0 is just as fast and saves effort!

In [ ]:
import time

np.random.seed(42)
x = np.linspace(0, 10, 100)
y = 5 * np.exp(-0.3 * x) + 2 + np.random.normal(0, 0.2, len(x))

# Method 1: Manual p0
start = time.time()
popt_manual, pcov_manual = curve_fit(
    functions.exponential_decay, x, y,
    p0=[5, 0.3, 2]  # User must guess!
)
time_manual = time.time() - start

# Method 2: Auto p0
start = time.time()
popt_auto, pcov_auto = curve_fit(
    functions.exponential_decay, x, y,
    p0='auto'  # Automatic!
)
time_auto = time.time() - start

print(f'Manual p0:   {popt_manual}')
print(f'Auto p0:     {popt_auto}')
print(f'\nDifference:  {np.max(np.abs(popt_manual - popt_auto)):.6f}')
print(f'\nTime (manual): {time_manual*1000:.2f}ms')
print(f'Time (auto):   {time_auto*1000:.2f}ms')
print('\n✓ Auto p0 is just as accurate but saves user effort!')

## 💡 Key Insights1. **Auto p0** eliminates tedious manual parameter guessing2. **Pre-built functions** cover most common use cases3. **Same accuracy** as manual p0 with less effort4. **Domain-specific features** like half-life, FWHM, EC50 calculations5. **Extensible** - easy to add custom functions---## 📚 Function Summary| Function | Form | Use Cases ||----------|------|-----------|| `linear` | ax + b | Baseline, calibration || `exponential_decay` | a·exp(-bx) + c | Radioactive decay, cooling || `exponential_growth` | a·exp(bx) + c | Population, compound interest || `gaussian` | a·exp(-(x-μ)²/(2σ²)) | Spectroscopy, chromatography || `sigmoid` | L/(1+exp(-k(x-x₀)))+b | Dose-response, growth || `power_law` | ax^b | Allometric scaling, physics || `polynomial(n)` | Σ aᵢx^i | General polynomial fits |---## 🎓 Next Steps- Use pre-built functions for your fitting tasks- Explore `.estimate_p0()` and `.bounds()` methods- Create custom functions following the same pattern- Combine with other NLSQ features (callbacks, error messages)---